In [ ]:
import os
import sys
sys.path.append("..")
import phage_init

In [ ]:
import pandas as pd
#d = {'model': [], 'class': [],'precision':[],'recall':[],'f1-score':[]}
d = {'model': [], 'class': [],'score_type':[],'value':[]}
df = pd.DataFrame(data=d)
F = open('all_models_table.txt','w') 

In [ ]:
#load the saved matrices
import pickle
train_X_total=pickle.load(open( os.path.join(phage_init.data_dir,"train_x.p"), "rb" ) )
test_X_total=pickle.load(open( os.path.join(phage_init.data_dir,"test_x.p"), "rb" ) )
train_Y=pickle.load(open( os.path.join(phage_init.data_dir,"train_y.p"), "rb" ) )
test_Y=pickle.load(open( os.path.join(phage_init.data_dir,"test_y.p"), "rb" ) )
mean_total=pickle.load(open( os.path.join(phage_init.data_dir,"mean.p"), "rb" ) )
std_total=pickle.load(open( os.path.join(phage_init.data_dir,"std.p"), "rb" ) )

In [ ]:
def add_to_df(df,test_Y_index, test_Y_predicted,model_name):
    labels_names=["Major capsid","Minor capsid","Baseplate","Major tail","Minor tail","Portal","Tail fiber",
             "Tail shaft","Collar","Head-Tail joining"]
    labels_dataframe=["Major capsid","Minor capsid","Baseplate","Major tail","Minor tail","Portal","Tail fiber",
                 "Tail shaft","Collar","Head-Tail joining","weighted avg"]
    for label in labels_dataframe:
        report=classification_report(test_Y_index, test_Y_predicted, target_names=labels_names,output_dict=True )
        #data_row=[report[label][i] for i in ['precision','recall',"f1-score"]]
        #data_row.insert(0,label)
        #data_row.insert(0,model_name)
        score_type='precision'
        data_row=[model_name,label,score_type,report[label][score_type]]
        df=df.append(pd.Series(data_row,index=df.columns),sort=False,ignore_index=True)
        score_type='recall'
        data_row=[model_name,label,score_type,report[label][score_type]]
        df=df.append(pd.Series(data_row,index=df.columns),sort=False,ignore_index=True)
        score_type='f1-score'
        data_row=[model_name,label,score_type,report[label][score_type]]
        df=df.append(pd.Series(data_row,index=df.columns),sort=False,ignore_index=True)
    return df
    


In [ ]:
di_train     = train_X_total[:,0:400]
tri_train    = train_X_total[:,400:8400]
di_sc_train  = train_X_total[:,8400:8449] 
tri_sc_train = train_X_total[:,8449:8792]
tt_train     = train_X_total[:,8792:]

In [ ]:
di_test     = test_X_total[:,0:400]
tri_test    = test_X_total[:,400:8400]
di_sc_test  = test_X_total[:,8400:8449] 
tri_sc_test = test_X_total[:,8449:8792]
tt_test     = test_X_total[:,8792:]

In [ ]:
test_Y_index = test_Y.argmax(axis=1) # Convert one-hot to index
labels_names=["Major capsid","Minor capsid","Baseplate","Major tail","Minor tail","Portal","Tail fiber",
             "Tail shaft","Collar","Head-Tail joining"]
labels_dataframe=["Major capsid","Minor capsid","Baseplate","Major tail","Minor tail","Portal","Tail fiber",
                 "Tail shaft","Collar","Head-Tail joining","weighted avg"]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Dropout
from keras.models import load_model
from sklearn.metrics import classification_report
import numpy

In [ ]:
test_X=di_test
model_name='di.h5'

model = load_model( os.path.join(phage_init.model_dir,model_name) )
test_Y_predicted = model.predict_classes(test_X)
print(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))
df=add_to_df(df,test_Y_index, test_Y_predicted,model_name)
F.write(model_name)
F.write(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))

In [ ]:
test_X=numpy.concatenate((di_test,tt_test),axis=1)
model_name='di_p.h5'

model = load_model( os.path.join(phage_init.model_dir,model_name) )
test_Y_predicted = model.predict_classes(test_X)
print(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))
df=add_to_df(df,test_Y_index, test_Y_predicted,model_name)
F.write(model_name)
F.write(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools
from collections import Counter
zz=Counter(test_Y_index)
sample_w=[zz[i] for i in range(0,10,1)]

CM=confusion_matrix(test_Y_index, test_Y_predicted)
CM_n=CM/numpy.array(sample_w)[:,None]
scale_up=1.3
plt.figure(figsize=[6.4*scale_up, 4.8*scale_up])
plt.imshow(CM_n, interpolation='nearest')
plt.title('CM')
plt.colorbar()
tick_marks = numpy.arange(len(labels_names))
plt.xticks(tick_marks, labels_names, rotation=90)
plt.yticks(tick_marks, labels_names)
fmt = '.2f'
for i, j in itertools.product(range(CM_n.shape[0]), range(CM_n.shape[1])):
        plt.text(j, i, format(CM_n[i, j], fmt),horizontalalignment="center",verticalalignment='center',
                color="white" if CM_n[i, j] < 0.25 else "black")

In [ ]:
test_X=tri_test
model_name='tri.h5'

model = load_model( os.path.join(phage_init.model_dir,model_name) )
test_Y_predicted = model.predict_classes(test_X)
print(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))
df=add_to_df(df,test_Y_index, test_Y_predicted,model_name)
F.write(model_name)
F.write(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))

In [ ]:
test_X=numpy.concatenate((tri_test,tt_test),axis=1)
model_name='tri_p.h5'

model = load_model( os.path.join(phage_init.model_dir,model_name) )
test_Y_predicted = model.predict_classes(test_X)
print(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))
df=add_to_df(df,test_Y_index, test_Y_predicted,model_name)
F.write(model_name)
F.write(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))

In [ ]:
test_X=di_sc_test
model_name='di_sc.h5'

model = load_model( os.path.join(phage_init.model_dir,model_name) )
test_Y_predicted = model.predict_classes(test_X)
print(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))
df=add_to_df(df,test_Y_index, test_Y_predicted,model_name)
F.write(model_name)
F.write(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))

In [ ]:
test_X=numpy.concatenate((di_sc_test,tt_test),axis=1)
model_name='di_sc_p.h5'

model = load_model( os.path.join(phage_init.model_dir,model_name) )
test_Y_predicted = model.predict_classes(test_X)
print(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))
df=add_to_df(df,test_Y_index, test_Y_predicted,model_name)
F.write(model_name)
F.write(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))

In [ ]:
test_X=tri_sc_test
model_name='tri_sc.h5'

model = load_model( os.path.join(phage_init.model_dir,model_name) )
test_Y_predicted = model.predict_classes(test_X)
print(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))
df=add_to_df(df,test_Y_index, test_Y_predicted,model_name)
F.write(model_name)
F.write(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))

In [ ]:
test_X=numpy.concatenate((tri_sc_test,tt_test),axis=1)
model_name='tri_sc_p.h5'

model = load_model( os.path.join(phage_init.model_dir,model_name) )
test_Y_predicted = model.predict_classes(test_X)
print(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))
df=add_to_df(df,test_Y_index, test_Y_predicted,model_name)
F.write(model_name)
F.write(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))

In [ ]:
test_X=test_X_total
model_name='all.h5'

model = load_model( os.path.join(phage_init.model_dir,model_name) )
test_Y_predicted = model.predict_classes(test_X)
print(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))
df=add_to_df(df,test_Y_index, test_Y_predicted,model_name)
F.write(model_name)
F.write(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
avg_df=df[df['class'] == 'weighted avg']

In [ ]:
fig, ax = plt.subplots()

fig.set_size_inches(18, 15)
sns.set(style="whitegrid")
ax.tick_params(axis='y',labelsize=16)
ax.tick_params(axis='x',labelsize=14)
ax.set_title('Weighted average model metrics', fontsize=40,va='bottom')
sns.barplot(ax=ax,y="value", x="model", hue="score_type", data=avg_df)
ax.set_ylabel('')    
ax.set_xlabel('')
l = ax.legend()
plt.setp(ax.get_legend().get_texts(), fontsize='22') # for legend text
#print(dir(l))
ax.set(ylim=(0, 1))
plt.show()
fig.savefig('avg_score')


In [ ]:
f1_df=df[df['score_type'] == 'f1-score']

In [ ]:
fig2, ax2 = plt.subplots()
fig2.set_size_inches(18, 15)
sns.set(style="whitegrid")
ax2.tick_params(axis='y',labelsize=16)
ax2.tick_params(axis='x',labelsize=14)
ax2.set_title('Per class model metrics', fontsize=40,va='bottom')
sns.barplot(ax=ax2,y="value", x="model", hue="class", data=f1_df)
ax2.set_ylabel('')    
ax2.set_xlabel('')
l = ax2.legend()
plt.setp(ax2.get_legend().get_texts(), fontsize='22') # for legend text
#print(dir(l))
ax2.set(ylim=(0, 1))
ax2.set(xlim=(-0.5, 11))
plt.show()
fig2.savefig('f1_score')

In [ ]:
F.close()